In [1]:
import gurobipy as gp
from gurobipy import GRB

 # Parameters

In [2]:
arcs, costs = gp.multidict({
    ('prod01', 'demand01'): 2,
    ('prod02', 'demand02'): 5,
    ('prod03', 'demand03'): 3,
    ('prod04', 'demand04'): 3,
    ('demand01', 'demand02'): 0.25,
    ('demand02', 'demand03'): 0.25,
    ('demand03', 'demand04'): 0.25,
})


supply = {
    'prod01': 600,
    'prod02': 300,
    'prod03': 500,
    'prod04': 400,
}

demand = {
    'demand01': 400,
    'demand02': 500,
    'demand03': 400,
    'demand04': 400,
}

# Model
## Decision Variables

In [3]:
m = gp.Model('inventory')
yards = m.addVars(arcs, vtype=GRB.CONTINUOUS, name='yards')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-19


## Objective Function

In [4]:
Z = yards.prod(costs)
m.ModelSense = GRB.MINIMIZE
m.setObjective(Z)

## Constraints

In [5]:
source = m.addConstrs(
    (yards.sum(i, '*') <= s for i,s in supply.items()),
    name='source'
)

transshipment = m.addConstrs(
    # In - Out = Demand   <--> In = Demand + Out
    (yards.sum('*', t) - yards.sum(t, '*')  == d for t,d in demand.items() ),
    name='inventory'
)

In [6]:
m.update()

print(f'Number of variables: {m.NumVars}')
print(f'Number of constraints: {m.NumConstrs}')
print(f'Constraints are \n {m.getConstrs()} \n')

Number of variables: 7
Number of constraints: 8
Constraints are 
 [<gurobi.Constr source[prod01,600]>, <gurobi.Constr source[prod02,300]>, <gurobi.Constr source[prod03,500]>, <gurobi.Constr source[prod04,400]>, <gurobi.Constr inventory[demand01,400]>, <gurobi.Constr inventory[demand02,500]>, <gurobi.Constr inventory[demand03,400]>, <gurobi.Constr inventory[demand04,400]>] 



## Optimization process

In [7]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 7 columns and 14 nonzeros
Model fingerprint: 0xc57615fa
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve removed 8 rows and 7 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.150000000e+03


## Results

In [8]:
print(f'Minimum Cost ${m.ObjVal:,.2f} \n')

for v in m.getVars():
    if abs(v.X) > 0.000001:
        print(f'{v.VarName}: {v.X}')

Minimum Cost $5,150.00 

yards[prod01,demand01]: 600.0
yards[prod02,demand02]: 300.0
yards[prod03,demand03]: 400.0
yards[prod04,demand04]: 400.0
yards[demand01,demand02]: 200.0
